### Preparing PySpark

In [1]:
import os
os.environ['SPARK_HOME'] = '/usr/local/lib/spark'
import findspark
findspark.init()
from pyspark import SparkContext
spark = SparkContext(appName='wordcount')
spark

<SparkContext master=local[*] appName=wordcount>

In [2]:
from pyspark import SQLContext
sqlc = SQLContext(spark)

In [4]:
# reading data
train = sqlc.read.csv("/home/ubuntu/.kaggle/competitions/avito-demand-prediction/train.csv",header=True)

In [52]:
# setting stopwords for russian text
from nltk.corpus import stopwords 
russian_stop = set(stopwords.words('russian'))

# Handle Text with SPARK

In [148]:
train = sqlc.read.csv("/home/ubuntu/.kaggle/competitions/avito-demand-prediction/train.csv",header=True,encoding="utf-8")

In [149]:
# get high deal probability data
trainHDP=train.filter(train["deal_probability"]>0.5).filter(train.description.isNotNull())
# get low deal probability data
trainLDP=train.filter(train["deal_probability"]<0.5).filter(train.description.isNotNull())

In [158]:
# define function to eliminate punctuations
import re
def nopunc(x):
    return re.sub(r'[^\w\s]',' ',x)

### transform text into list, and get list aggragation

In [167]:
mylistH=trainHighDP.rdd \
    .flatMap(lambda x:x["description"].replace(","," ").replace("."," ").split(' ')) \
    .filter(lambda x:x.lower() not in russian_stop) \
    .filter(lambda x: (len(x)>2)) \
    .map(lambda t: (t.lower(), 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(100, key=lambda pair: -pair[1])

In [ ]:
mylistL=trainLowDP.rdd \
    .flatMap(lambda x:x["description"].replace(","," ").replace("."," ").split(' ')) \
    .filter(lambda x:x.lower() not in russian_stop) \
    .filter(lambda x: (len(x)>2)) \
    .map(lambda t: (t.lower(), 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .takeOrdered(100, key=lambda pair: -pair[1])

### get aggragation to a dictionary and get all keywords

In [160]:
mydictH=dict(mylistH)
mydictL=dict(mylistL)

In [161]:
mydictH

{'100': 14835,
 'большой': 10609,
 'ботинки': 9633,
 'бу': 31986,
 'внутри': 11143,
 'вода': 10087,
 'возможен': 17381,
 'вопросы': 15922,
 'время': 9712,
 'высота': 9840,
 'газ': 9191,
 'гараж': 9594,
 'года': 23167,
 'города': 10011,
 'две': 10340,
 'девочки': 15554,
 'девочку': 11749,
 'джинсы': 10067,
 'длина': 19949,
 'документы': 11108,
 'дом': 31128,
 'дома': 16430,
 'доме': 13958,
 'звоните': 17237,
 'идеальное': 12212,
 'идеальном': 21527,
 'имеется': 12674,
 'качество': 10044,
 'квартира': 29756,
 'квартиру': 21038,
 'кожа': 20252,
 'комбинезон': 12263,
 'комнаты': 11473,
 'комплект': 15257,
 'комплекте': 13651,
 'костюм': 13989,
 'куртка': 17522,
 'кухня': 11658,
 'лет': 21280,
 'магазины': 9365,
 'мало': 10241,
 'мальчика': 17531,
 'машина': 9565,
 'наличии': 11702,
 'натуральная': 14094,
 'новая': 34478,
 'новое': 15663,
 'новые': 47895,
 'новый': 42567,
 'носили': 13830,
 'обмен': 12740,
 'окна': 16282,
 'отдам': 12784,
 'отличное': 36119,
 'отличном': 71934,
 'очень': 77

In [4]:
dealY=list(mydictH.keys())
dealN=list(mydictL.keys())

#lists of top words 

#top words for deal_probability >= 0.5
dealY = ('состоянии', 'продам', 'хорошем', 'состояние', 'отличном', 'торг', 'очень',     
         'продаю', 'размер', 'цена', 'б/у', 'работает', 'новый', 'хорошее', 'отличное', 
         'телефону', 'года', 'фото', 'новые', 'отдам', 'новая', 'комплекте', 'машина', 
         'продается', 'вопросы', 'цвет', 'ремонт', 'лет', 'квартира', 'идеальном', 
         'квартиру', 'комплект', 'работы', 'авто', 'руб', 'автомобиль', 'рабочем', 
         'коробка', 'телефон', 'срок', 'сдам', 'возможен', 'звоните', 'полностью', 
         'обмен', 'дом', 'коляска', 'велосипед', '"продам', 'двигатель', 'длительный', 
         'срочно', 'подарок', 'имеется', 'самовывоз', 'полный', 'холодильник', 'резина', 
         'музыка', 'время', 'сигнализация', 'диван', 'длина', 'дома', 'салон', 'всё', 
         'связи', 'документы', 'любой', 'отлично', 'рублей', 'высота', 'продаётся', 'чехол', 
         'колеса', 'мальчика', 'большой', 'детский', 'требует', 'рядом', 'стоит', 'девочки', 
         'пробег', 'кожа', 'год', 'фирмы', 'идеальное', 'коляску', 'мало', 'мебель', 'также', 
         'ходу', 'цвета', 'уместен', 'услуги', 'птс', 'торга', 'рост', 'оригинал', 'стол')

#top words for deal_probability < 0.5
dealN = ('состоянии', 'продам', 'размер', 'состояние', 'хорошем', 'очень', 'отличном',   
         'торг', 'продаю', 'новые', 'цена', 'новый', 'отличное', 'новая', 'хорошее', 'платье', 
         'дом', 'б/у', 'квартира', 'цвет', 'продается', 'руб', 'фото', 'рост', 'года', 
         'идеальном', 'квартиру', 'лет', 'рядом', 'кожа', 'телефону', 'длина', 'участок', 
         'цвета', 'мальчика', 'возможен', 'ремонт', 'куртка', 'окна', 'дома', 'вопросы', 
         'девочки', 'работает', 'новое', 'звоните', 'комплект', 'рублей', 'натуральная', 
         'костюм', 'доме', 'носили', 'пару', 'комплекте', 'подарок', 'отдам', 'обмен', 'имеется', 
         'комбинезон', 'туфли', 'телефон', 'девочку', 'идеальное', 'районе', 'пальто', 'кухня', 
         'комнаты', 'уместен', 'наличии', 'внутри', 'документы', 'район', 'стельке', 'продаётся', 
         'большой', 'фирмы', 'школа', 'две', 'срочно', 'полностью', 'мало', 'джинсы', 'вода', 
         'раза', 'качество', 'высота', 'города', 'сапоги', 'ботинки', 'машина', 'гараж', 'покупала', 
         'удобные', 'магазины', 'покупали', 'работы', 'также', 'время', 'газ', 'пишите', 'детский',)

In [ ]:
#eliminiate words that are in both lists
dealY_unique = [elem for elem in dealY if elem not in dealN]
dealN_unique = [elem for elem in dealN if elem not in dealY]

In [ ]:
#create and apply function to count yes-deal words for all entries in 'description'
wcf = lambda d: [sum(d.count(x) for x in dealY_unique)]
train['wordcountY'] = train['description'].apply(wcf)

#create and apply function to count no-deal words for all entries in 'description'
wcf = lambda d: [sum(d.count(x) for x in dealN_unique)]
train['wordcountN'] = train['description'].apply(wcf)

##### Similarly, by changing file name and path we can do analysis to test data.